In [2]:
!pip install textblob

     |████████████████████████████████| 645kB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 1.4MB 2.4MB/s eta 0:00:01
     |████████████████████████████████| 92kB 3.8MB/s eta 0:00:01
     |████████████████████████████████| 675kB 3.4MB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk


In [4]:
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas
import sklearn
import numpy as np

import nltk
# nltk.download('popular')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split 

In [6]:
sms = pandas.read_csv('./datasets/sms_spam_no_header.csv', sep=',', names=["type", "text"])

In [11]:
from textblob import TextBlob

def get_tokens(msg):
    return TextBlob(str(msg)).words

def get_lemmas(msg):
    lemmas = []
    words  = get_tokens(msg)
    
    for word in words:
        lemmas.append(word.lemma)

    return lemmas


# Thanks to Sebastian Raschka for 'plot_decision_regions'
# https://github.com/rasbt/python-machine-learning-book

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import warnings

def versiontuple(v):
    return tuple(map(int, (v.split("."))))

def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=cmap(idx),
                    marker=markers[idx], label=cl)

    # highlight test samples
    if test_idx:
        # plot all samples
        if not versiontuple(np.__version__) >= versiontuple('1.9.0'):
            X_test, y_test = X[list(test_idx), :], y[list(test_idx)]
            warnings.warn('Please update to NumPy 1.9.0 or newer')
        else:
            X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c='',
                    alpha=1.0,
                    linewidths=1,
                    marker='o',
                    s=55, label='test set')

In [12]:
text_train, text_test, type_train, type_test = train_test_split(sms['text'], sms['type'], test_size=0.3)

In [14]:
! python -m textblob.download_corpora


[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [15]:
bow = CountVectorizer(analyzer=get_lemmas).fit(text_train)

In [16]:
sms_bow = bow.transform(text_train)

In [17]:
tfidf = TfidfTransformer().fit(sms_bow)

In [18]:
sms_tfidf = tfidf.transform(sms_bow)

In [19]:
spam_detector = MultinomialNB().fit(sms_tfidf, type_train)

In [20]:
msg = sms['text'][25]
msg_bow = bow.transform([msg])
msg_tfidf = tfidf.transform(msg_bow)

print ('predicted:', spam_detector.predict(msg_tfidf)[0])
print ('expected:', sms.type[25])

predicted: ham
expected: ham


In [21]:
predictions = spam_detector.predict(sms_tfidf)

In [22]:
print ('accuracy', accuracy_score(sms['type'][:len(predictions)], predictions))

accuracy 0.7928736221481671


In [23]:
print (classification_report(sms['type'][:len(predictions)], predictions))

              precision    recall  f1-score   support

         ham       0.87      0.90      0.88      3382
        spam       0.14      0.10      0.12       519

    accuracy                           0.79      3901
   macro avg       0.50      0.50      0.50      3901
weighted avg       0.77      0.79      0.78      3901

